## Installation and Imports

In [1]:
!pip install sunpy==3.1.0
!pip install matplotlib==3.1.3
!pip install git+https://github.com/RobertJaro/InstrumentToInstrument.git
!pip uninstall dem -y
!pip install git+https://github.com/RobertJaro/DeepEM.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 21.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 13.9 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/RobertJaro/InstrumentToInstrument.git to /tmp/pip-req-build-bgaui2_i
  Running command git clone --filter=blob:none --quiet https://github.com/RobertJaro/InstrumentToInstrument.git /tmp/pip-req-build-bgaui2_i
  Resolved https://github.com/RobertJaro/InstrumentToInstrument.git to commit 523176a59c8b4239df53a81992400a12dd4736e2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.7/500.7 KB 9.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 116.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/RobertJaro/DeepEM.git to /tmp/pip-req-build-_y3dt99v
  Running command git clone --filter=blob:none --quiet https://github.com/RobertJaro/DeepEM.git /tmp/pip-req-build-_y3dt99v
  Resolved https://github.com/RobertJaro/DeepEM.git to commit beb866b7e32e6167a9f40f9c1e6912746c8974f4
  Preparing metadata (setup.py) ... done
  Created wheel for DeepEM: filename=DeepEM-0.1-py3-none-any.whl size=36209 sha256=b72bca407bf3a9278e89ca27efd3dd04d5194fba3b3d0aea31e674e262dd86cc
  Stored in directory: /tmp/pip-ephem-wheel-cache-l0d55rcb/wheels/5d/08/b0/1444fb7f69c3df9405fb7d9d154ea82750d82ed547f66ca6de
Successfully built DeepEM


In [16]:
# imports for download
import os
import shutil
import drms
from datetime import datetime, timedelta

# DEM tool
from dem.train.model import DEM
from dem.train.generator import AIADEMDataset

# data processing
import numpy as np
from skimage.measure import block_reduce
from astropy import units as u
from astropy.coordinates import SkyCoord
from sunpy.coordinates import frames
from torch.utils.data import DataLoader
from astropy.io import fits

# visualization
from sunpy.map import Map
from matplotlib import pyplot as plt
from astropy.visualization import ImageNormalize, AsinhStretch
from sunpy.visualization.colormaps import cm
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

# file zip
import gzip
import glob

In [3]:
#@title General Settings
evaluation_path = 'evaluation' #@param {type:"string"}
fits_path = 'fits' #@param {type:"string"}

os.makedirs(evaluation_path, exist_ok=True)
os.makedirs(fits_path, exist_ok=True)

In [4]:
#@title Download Settings
download_dir = 'sdo_data' #@param {type:"string"}

#@markdown Downloading data requires an active registration at JSOC. http://jsoc.stanford.edu/ajax/register_email.html (free of charge)
email = 'robert.jarolim@uni-graz.at' #@param {type:"string"}

# initialize the download client and directories
wls = ['94', '131', '171', '193', '211', '335']
[os.makedirs(os.path.join(download_dir, wl), exist_ok=True) for wl in wls]
client = drms.Client(email=email, verbose=True)

In [5]:
#@title Data Selection
year = 2014 #@param {type:"integer"}
month = 1 #@param {type:"integer"}
day = 1 #@param {type:"integer"}
hour = 18 #@param {type:"integer"}
minute = 35 #@param {type:"number"}
#
duration = '1h' #@param {type: 'string'}
cadence = '12s' #@param {type:"string"}
locunits = "arcsec" #@param {type: 'string'}
boxunits = "pixels" #@param {type: 'string'}
#
cutout = True #@param {type: 'boolean'}
x = 675 #@param {type: 'number'}
y = -225 #@param {type: 'number'}
width = 256 #@param {type: 'number'}
height = 256 #@param {type: 'number'}

# create datetime object
start_date = datetime(year, month, day, hour, minute)

In [6]:
# define cutout
if cutout:
  process = {
      "im_patch": {
          "t_ref": start_date.isoformat('T'),
          "t": 0,
          "r": 0,
          "c": 0,
          "locunits": locunits,
          "boxunits": boxunits,
          "x": x,
          "y": y,
          "width": width,
          "height": height,
      }
  }
else:
  process = None

# query data
qstr = 'aia.lev1_euv_12s[%s/%s@%s][%s]{image}' % (start_date.isoformat('T'), duration, cadence, ','.join(wls) )
r = client.export(qstr, method="url", protocol="fits", process=process)
r.wait()

# download files
downloaded_files = r.download(download_dir)
# distribute to folders
for f in downloaded_files.download:
  path_elements = os.path.basename(f).split('.')
  f_date = path_elements[2]
  wl = path_elements[3]
  shutil.move(f, os.path.join(download_dir, wl, f_date[:-1] + '.fits'))

Streaming output truncated to the last 5000 lines.
    record: aia.lev1_euv_12s_mod[2014-01-01T18:52:37Z][193][JSOC_20230127_993]
  filename: aia.lev1_euv_12s.2014-01-01T185237Z.193.image.fits
  -> sdo_data/aia.lev1_euv_12s.2014-01-01T185237Z.193.image.fits
    record: aia.lev1_euv_12s_mod[2014-01-01T18:52:37Z][211][JSOC_20230127_993]
  filename: aia.lev1_euv_12s.2014-01-01T185237Z.211.image.fits
  -> sdo_data/aia.lev1_euv_12s.2014-01-01T185237Z.211.image.fits
    record: aia.lev1_euv_12s_mod[2014-01-01T18:52:37Z][335][JSOC_20230127_993]
  filename: aia.lev1_euv_12s.2014-01-01T185237Z.335.image.fits
  -> sdo_data/aia.lev1_euv_12s.2014-01-01T185237Z.335.image.fits
    record: aia.lev1_euv_12s_mod[2014-01-01T18:52:49Z][94][JSOC_20230127_993]
  filename: aia.lev1_euv_12s.2014-01-01T185249Z.94.image.fits
  -> sdo_data/aia.lev1_euv_12s.2014-01-01T185249Z.94.image.fits
    record: aia.lev1_euv_12s_mod[2014-01-01T18:52:49Z][131][JSOC_20230127_993]
  filename: aia.lev1_euv_12s.2014-01-01T18524

## Compute DEM maps

Initialize model for estimating the DEM maps.

In [7]:
dem_model = DEM()
logT = dem_model.log_T
T = 10 ** logT

In [8]:
# data loader
ds = AIADEMDataset(download_dir, skip_register=True)
loader = DataLoader(ds, batch_size=None, num_workers=4)

# load reference dates
ref_map_paths = sorted(glob.glob(os.path.join(download_dir, '94', '*.fits')))

/usr/local/lib/python3.8/dist-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/usr/local/lib/python3.8/dist-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/usr/local/lib/python3.8/dist-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 100 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/usr/local/lib/python3.8/dist-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in curre

In [ ]:
for idx, (image, ref_map_path) in enumerate(zip(loader, ref_map_paths)):
    image = image.detach().numpy()
    dem_result = dem_model.compute_patches(image, (256, 256), uncertainty=False)

    dem = dem_result['dem']
    dem_uncertainty = dem_result['dem_uncertainty'] if 'dem_uncertainty' in dem_result else np.zeros_like(dem)
    reconstruction = dem_result['reconstruction']

    # load ref map
    ref_map = Map(ref_map_path)
    date_str = ref_map.date.to_datetime().isoformat('T')

    # PLOT result
    fig = plt.figure(figsize=(8, 4), constrained_layout=True)
    gs = fig.add_gridspec(2, 3)
    # plot EUV image
    ax = fig.add_subplot(gs[0, 0])
    ax.set_axis_off()
    ax.imshow(image[3], cmap=cm.sdoaia193, norm=ImageNormalize(vmin=-1, vmax=1, stretch=AsinhStretch(0.005)), origin='lower')

    total_DEM = (dem * np.gradient(T[:, None, None], axis=0)).sum(0)
    mean_T = (dem * T[:, None, None] * np.gradient(T[:, None, None], axis=0)).sum(0) / (total_DEM + 1e-6)
    # plot average temperature
    ax = fig.add_subplot(gs[0, 1])
    ax.set_axis_off()
    T_im = ax.imshow(mean_T * 1e-6, cmap='inferno', origin='lower', vmin=4, vmax=10)
    T_divider = make_axes_locatable(ax)
    cT = T_divider.append_axes("right", size="7%", pad="2%")
    cbT = fig.colorbar(T_im, cax=cT, label='Temperatur [MK]')
    # plot total DEM
    ax = fig.add_subplot(gs[0, 2])
    ax.set_axis_off()
    dem_im = ax.imshow(total_DEM, cmap='viridis', origin='lower', vmin=1e27, vmax=1e29)
    T_divider = make_axes_locatable(ax)
    cT = T_divider.append_axes("right", size="7%", pad="2%")
    cbT = fig.colorbar(dem_im, cax=cT, label='Total EM [cm$^-5$]')
    # plot DEM distribution
    ax = fig.add_subplot(gs[1, :])
    ax.set_title(date_str)
    ax.errorbar(T * 1e-6, dem.mean((1, 2)), yerr=dem_uncertainty.mean((1, 2)), ecolor='red', capsize=2)
    ax.set_xlim(0, 25)
    ax.set_ylim(1e19, 5e21)
    ax.set_xlabel('T [MK]')
    ax.set_ylabel('DEM [cm$^{-5}$ K$^{-1}$]')
    ax.semilogy()
    fig.tight_layout()
    fig.savefig(os.path.join(evaluation_path, f'%s.jpg' % date_str), dpi=300)
    plt.close(fig)
    plt.close('all')
    #
    for temp, dem_bin, dem_uc in zip(logT, dem, dem_uncertainty):
      meta_info = {'DATE-OBS': date_str, 'TEMPERATURE': temp}
      # save FITS
      fp = os.path.join(fits_path, '%s_TEMP%.02f.fits' % (date_str, temp))
      if os.path.exists(fp):
        os.remove(fp)
      t_map = Map(dem_bin, ref_map.wcs)
      for i, v in meta_info.items():
        t_map.meta[i] = v
      t_map.save(fp)
      # gz FITS
      with open(fp, 'rb') as f_in, gzip.open(fp + '.gz', 'wb') as f_out:
        f_out.writelines(f_in)
      os.remove(fp)


## Show Video

In [23]:
from IPython.display import HTML
from base64 import b64encode
import cv2

In [24]:
#@title Video Settings
video_name = 'video.mp4' #@param {type:"string"}

In [27]:
images = sorted(glob.glob(os.path.join(evaluation_path, '*.jpg')))
frame = cv2.imread(images[0])
height, width, layers = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(video_name, fourcc, 10, (width,height))

for image in images:
    video.write(cv2.imread(image))

cv2.destroyAllWindows()
video.release()

In [13]:
from IPython.display import Video
Video(video_name)